In [1]:
!python3 -m pip install --upgrade -q boto3
!python3 -m pip install --upgrade -q botocore
!python3 -m pip install --upgrade -q awscli

In [ ]:
!python3 -m pip install pandas

In [ ]:
!python3 -m pip install numpy

In [1]:
import boto3
import botocore
import awscli
print(boto3.__version__)
print(botocore.__version__)
print(awscli.__version__)

1.38.24
1.38.24
1.40.23


In [2]:
import json
import time
from io import BytesIO
import uuid
import pprint
import logging

In [3]:
# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [40]:
# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client("bedrock-agent", region_name="us-east-1")
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region_name="us-east-1")

In [ ]:
import boto3

session = boto3.Session(
    profile_name='default',
    region_name='us-east-1' 
)

sts_client = session.client('sts')
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]

region, account_id

In [49]:
# configuration variables
suffix = f"{region}-{account_id}"
agent_name = "assistant-w-code-interpret"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}"
agent_role_name = f"AmazonBedrockExecutionRoleAgents_{agent_name}"
agent_foundational_model = "amazon.nova-pro-v1:0"
agent_description = "Assistant with code interpreter that can write and execute code to answer questions"
agent_instructions = """ You are an assistant that helps customers answer questions and create documents.
You have access to code interpreter to execute Python code, so when tasks are best handled via Python code, 
write code as needed and pass it to code interpreter to execute, then return the result to the user.
"""
agent_alias_name = f"{agent_name}-alias"

### Create synthetic stock price data

In [12]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def make_synthetic_stock_data(filename):

    start_date = datetime(2023, 6, 27)
    end_date = datetime(2024, 6, 27)

    date_range = pd.date_range(start_date, end_date, freq='D')

    symbol = []
    dates = []
    open_prices = []
    close_prices = []
    high_prices = []
    low_prices = []
    adj_close_prices = []
    volumes = []

    initial_price = 100.0

    for date in date_range:
        symbol.append('FAKECO')
        dates.append(date)
        open_price = np.round(initial_price + np.random.uniform(-1, 1), 2)
        high_price = np.round(open_price + np.random.uniform(0, 5), 2)
        low_price = np.round(open_price - np.random.uniform(0, 5), 2)
        close_price = np.round(np.random.uniform(low_price, high_price), 2)
        adj_close_price = close_price
        volume = np.random.randint(1000, 10000000)

        open_prices.append(open_price)
        high_prices.append(high_price)
        low_prices.append(low_price)
        close_prices.append(close_price)
        adj_close_prices.append(adj_close_price)
        volumes.append(volume)

        initial_price = close_price
    
    data = {
        'Symbol': symbol,
        'Date': dates,
        'Open': open_prices,
        'Close': close_prices,
        'High': high_prices,
        'Low': low_prices,
        'Adj Close': close_prices,
        'Volume': volumes
    }

    stock_data = pd.DataFrame(data)

    stock_data.to_csv(filename, index=False)

In [66]:
import os

if not os.path.exists('output'):
    os.mkdir('output')

stock_file = os.path.join('output', 'FAKECO.csv')

if not os.path.exists(stock_file):
    make_synthetic_stock_data(stock_file)

### Create IAM Policy and Role

In [50]:
# Create IAM policies for agent

bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{agent_foundational_model}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)


In [ ]:
assume_role_policy_document = assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

#### Creating the Bedrock agent

In [ ]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundational_model,
    instruction=agent_instructions
)
response

In [ ]:
agent_id = response['agent']['agentId']
agent_id

#### Create Agent Action Group

In [54]:
# Pause to make sure agent is created
#time.sleep(30)
# Now, we can configure and create an action group here:

# Enable code interpretation for the agent
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,       
    agentVersion='DRAFT',
    actionGroupName='code-interpreter',
    parentActionGroupSignature='AMAZON.CodeInterpreter',
    actionGroupState='ENABLED'
)

In [ ]:
agent_action_group_response

#### Preparing Agent

In [ ]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)

In [57]:
# Pause to make sure agent is prepared
#time.sleep(30)

# Extract the agentAliasId from the response
agent_alias_id = "TSTALIASID"

#### Invoking the agent

In [58]:
def process_response(resp, enable_trace: bool = False, show_code_use: bool = False):
    if enable_trace:
        logger.info(pprint.pformat(resp))

    event_stream = resp['completion']

    try:
        for event in event_stream:
            if 'chunk' in event:
                data = event['chunk']['bytes']
                if enable_trace:
                    logger.info(f"Final Answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                return agent_answer

            elif 'trace' in event:
                trace_data = event['trace']

                # Check presence of code interpreter without serializing
                if (
                    isinstance(trace_data, dict)
                    and 'codeInterpreterInvocationInput' in trace_data
                ):
                    if show_code_use:
                        print("Invoked Code Interpreter")
                    if enable_trace:
                        # Use safe serialization for logging
                        logger.info(json.dumps(trace_data, default=str, indent=2))

            else:
                raise Exception("unexpected event.", event)

    except Exception as e:
        raise Exception("unexpected event.", e)

In [59]:
def invoke_agent_helper(
       query, session_id, agent_id, alias_id, enable_trace=False, memory_id=None, session_state=None, end_session=False, show_code_use=False 
):
    if not session_state:
        session_state = {}
    
    # invoke the agent API
    agent_response = bedrock_agent_runtime_client.invoke_agent(
        inputText=query,
        agentId=agent_id,
        agentAliasId=alias_id,
        sessionId=session_id,
        enableTrace=(enable_trace | show_code_use),
        sessionState=session_state,
        endSession=end_session,
        memoryId=memory_id
    )

    return process_response(agent_response, enable_trace=enable_trace, show_code_use=show_code_use)


In [ ]:
agent_aliases = bedrock_agent_client.list_agent_aliases(agentId=agent_id)
print(agent_aliases)

#### Invoking Code Interpreter

In [ ]:
## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
memory_id:str = 'TST_MEM_ID'
query = "Please generate a 10 character long string of random characters"
invoke_agent_helper(query, session_id, agent_id, agent_alias_id, enable_trace=False, memory_id=memory_id, show_code_use=True)

In [ ]:
query = "What is 75 * sin(.75)?"
invoke_agent_helper(query, session_id, agent_id, agent_alias_id, enable_trace=False, memory_id=memory_id, 
                    show_code_use=True)

In [ ]:
query = "thank you!"
invoke_agent_helper(query, session_id, agent_id, agent_alias_id, enable_trace=False, memory_id=memory_id, show_code_use=True)

#### Sending files to the agent

In [64]:
def add_file_to_session_state(file_name, use_case='CODE_INTERPRETER', session_state=None):
    if use_case != "CHAT" and use_case != "CODE_INTERPRETER":
        raise ValueError("Use case must be either 'CHAT' or 'CODE_INTERPRETER'")
    if not session_state:
        session_state = {
            "files": []
        }
    type = file_name.split(".")[-1].upper()
    name = file_name.split("/")[-1]

    if type == "CSV":
        media_type = "text/csv" 
    elif type in ["XLS", "XLSX"]:
        media_type = "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
    else:
        media_type = "text/plain"

    named_file = {
        "name": name,
        "source": {
            "sourceType": "BYTE_CONTENT", 
            "byteContent": {
                "mediaType": media_type,
                "data": open(file_name, "rb").read()
            }
        },
        "useCase": use_case
    }
    session_state['files'].append(named_file)

    return session_state

#### Passing files for normal chat

In [ ]:
import base64 

# base64 encode the csv file 
with open(stock_file, "rb") as file_name:
    data = file_name.read()
    encoded_file = data #base64.b64encode(data)

    # Show the first 100 characters of the encoded file
encoded_file[0:100]

In [ ]:
# Invoke the agent and process the response stream
query = "What is the data in this file?"

sessionState=add_file_to_session_state(stock_file, 'CHAT')

invoke_agent_helper(query, session_id, agent_id, agent_alias_id, enable_trace=False, session_state=sessionState, 
                    memory_id=memory_id, show_code_use=True)

#### Passing files for use with code interpretation

In [ ]:
# Invoke the agent and process the response stream
query = "Given the attached price data file, what pct growth happened across the full time series for closing price? what was the price on the first and last days?"

sessionState=add_file_to_session_state(stock_file, 'CODE_INTERPRETER')

invoke_agent_helper(query, session_id, agent_id, agent_alias_id, enable_trace=False, session_state=sessionState, 
                    memory_id=memory_id, show_code_use=True)

### The Bedrock Agents response stream

The response stream consists of events, formatted in JSON. It conveys rich data about the details of the agent's thought and actions as it works through the ReAct pattern (reasoning and action). Here are some important keys:

- 'files' contain files generated by the agent's LLM model intrinsically
- 'trace' events contain information about the agent's thought process and work steps. There are several kinds of trace events:
- 'modelInvocationInput' keys contain
- 'rationale' keys contain the agent's reasoning
- 'invocationInput' keys contain details of parameters to action group calls.
- 'codeInterpreterInvocationInput' keys within that contain code that the model generated and is passing to code interpretation.
- 'observation' keys contain important observations, including:
- 'codeInterpreterInvocationOutput' within that contains specific output from the code interpretation:
- 'executionOutput' contains the results of the code execution
- 'executionError' is populated with an error if an error is encountered while executing the code
- 'files' contain files generated by the code interpretation
- 'finalResponse' contains the agent's final response

### Redefine the helper function


In [ ]:
!python3 -m pip install matplotlib

In [ ]:
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


In [73]:
def process_response(resp, enable_trace:bool=True, show_code_use:bool=False):
    if resp['ResponseMetadata']['HTTPStatusCode'] != 200:
        print(f"API Response was not 200: {resp}")

    event_stream = resp['completion']
    for event in event_stream:
        if 'files' in event.keys():
            files_event = event['files']
            display(Markdown("### Files"))
            files_list = files_event['files']
            for this_file in files_list:
                print(f"{this_file['name']} ({this_file['type']})")
                file_bytes = this_file['bytes']

                # save bytes to file, given the name of file and the bytes 
                file_name = os.path.join('output', this_file['name'])
                with open(file_name, 'wb') as f:
                    f.write(file_bytes)
                if this_file['type'] == 'image/png' or this_file['type'] == 'image/jpeg':
                    img = mpimg.imread(file_name)
                    plt.imshow(img)
                    plt.show()

        if 'trace' in event.keys() and enable_trace:
            trace_event = event.get('trace')['trace']['orchestrationTrace']

            if 'modelInvocationInput' in trace_event.keys():
                pass

            if 'rationale' in trace_event.keys():
                rationale = trace_event['rationale']['text']
                display(Markdown(f"### Rationale\n{rationale}"))

            if 'invocationInput' in trace_event.keys() and show_code_use:
                inv_input = trace_event['invocationInput']
                if 'codeInterpreterInvocationInput' in inv_input:
                    gen_code = inv_input['codeInterpreterInvocationInput']['code']
                    code = f"```python\n{gen_code}\n```"
                    display(Markdown(f"### Generated code\n{code}"))

            if 'observation' in trace_event.keys():
                obs = trace_event['observation']
                if 'codeInterpreterInvocationOutput' in obs:
                    if 'executionOutput' in obs['codeInterpreterInvocationOutput'].keys() and show_code_use:
                        raw_output = obs['codeInterpreterInvocationOutput']['executionOutput']
                        output = f"```\n{raw_output}\n```"
                        display(Markdown(f"### Output from code execution\n{output}"))

                    if 'executionError' in obs['codeInterpreterInvocationOutput'].keys():
                        display(Markdown(f"### Error from code execution\n{obs['codeInterpreterInvocationOutput']['executionError']}"))

                    if 'files' in obs['codeInterpreterInvocationOutput'].keys():
                        display(Markdown("### Files generated\n"))
                        display(Markdown(f"{obs['codeInterpreterInvocationOutput']['files']}"))

                if 'finalResponse' in obs:                    
                    final_resp = obs['finalResponse']['text']
                    display(Markdown(f"### Final response\n{final_resp}"))
                    return final_resp

### Generate a file using code generation

In [ ]:
query = """
Please generate a list of the 10 greatest books of all time. Return it as a CSV file. Always return the file, even if you have provided it before.
"""

invoke_agent_helper(query, session_id, agent_id, agent_alias_id, enable_trace=False, session_state=sessionState,
                    memory_id=memory_id, show_code_use=True)

#### Generate a chart using code interpretation

In [ ]:
# Invoke the agent and process the response stream
query = "Given the attached price data file, please make me a chart with moving average in red and actual data in blue"

sessionState=add_file_to_session_state(stock_file, 'CODE_INTERPRETER')

invoke_agent_helper(query, session_id, agent_id, agent_alias_id, enable_trace=True, session_state=sessionState,
                    memory_id=memory_id, show_code_use=True)

#### Generate synthetic data and analyze it

In [ ]:
# Invoke the agent and process the response stream
query = """
Generate two CSV files for me: 
1. One called SALES, with 3 columns: COMPANY_ID, COMPANY_NAME, and SALES_2024.  
2. Another called DETAILS, with 2 columns: COMPANY_ID and COMPANY_STATE_CODE.

Follow these rules:
- Each file should contain data for 200 companies, and both must share the same COMPANY_IDs.
- COMPANY_IDs should be of the form: C00001, C00002, ..., C00200.
- Use realistic, human-readable English words for company names (not random characters).
- Company names should be generated without using nested quotes in f-strings — assign choices to variables before using them in f-strings.
- States should be chosen only from U.S. East Coast or West Coast states (e.g., NY, NJ, MA, CA, WA).
- Assign 100 companies to East Coast states and 100 to West Coast states.
- SALES_2024 should range between 0–700,000 for East Coast companies, and 500,000–2,000,000 for West Coast companies.

Once the CSV files are generated, merge the data and produce a boxplot comparing SALES_2024 between East Coast and West Coast companies. Save the chart as a PNG.

Important:
- Avoid syntax errors. Do not use nested quote expressions directly inside f-strings.
- Make sure the files are saved locally as './SALES.csv', './DETAILS.csv', and './sales_comparison.png'.
- After generating, validate that all rules were followed.

When done, test to be sure you have followed each of the above rules, 
and produce a chart comparing sales per company in the two regions using box plots.
"""

invoke_agent_helper(query, session_id, agent_id, agent_alias_id, enable_trace=True, session_state=sessionState,
                    memory_id=memory_id, show_code_use=True)

### Clean up

In [83]:
action_group_id = agent_action_group_response['agentActionGroup']['actionGroupId']
action_group_name = agent_action_group_response['agentActionGroup']['actionGroupName']

response = bedrock_agent_client.update_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id,
    actionGroupName=action_group_name,
    actionGroupState='DISABLED',
    parentActionGroupSignature='AMAZON.CodeInterpreter'
)

In [84]:
action_group_deletion = bedrock_agent_client.delete_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id
)
agent_deletion = bedrock_agent_client.delete_agent(
    agentId=agent_id
)

In [ ]:

# Delete IAM Roles and policies

for policy in [agent_bedrock_allow_policy_name]:
    iam_client.detach_role_policy(RoleName=agent_role_name, PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}')

for policy in [agent_bedrock_policy]:
    iam_client.delete_policy(
        PolicyArn=policy['Policy']['Arn']
)
    
iam_client.delete_role(
    RoleName=agent_role_name
)